# Problem 61

### Cyclical figurate numbers

Triangle, square, pentagonal, hexagonal, heptagonal, and octagonal numbers are all figurate (polygonal) numbers and are generated by the following formulae:

    Triangle	 	P3,n=n(n+1)/2	 	1, 3, 6, 10, 15, ...
    Square	 	P4,n=n^2	 	1, 4, 9, 16, 25, ...
    Pentagonal	 	P5,n=n(3n−1)/2	 	1, 5, 12, 22, 35, ...
    Hexagonal	 	P6,n=n(2n−1)	 	1, 6, 15, 28, 45, ...
    Heptagonal	 	P7,n=n(5n−3)/2	 	1, 7, 18, 34, 55, ...
    Octagonal	 	P8,n=n(3n−2)	 	1, 8, 21, 40, 65, ...
    
The ordered set of three 4-digit numbers: 8128, 2882, 8281, has three interesting properties.

The set is cyclic, in that the last two digits of each number is the first two digits of the next number (including the last number with the first).
Each polygonal type: triangle (P3,127=8128), square (P4,91=8281), and pentagonal (P5,44=2882), is represented by a different number in the set.
This is the only set of 4-digit numbers with this property.
Find the sum of the only ordered set of six cyclic 4-digit numbers for which each polygonal type: triangle, square, pentagonal, hexagonal, heptagonal, and octagonal, is represented by a different number in the set.

### Solution

First we generate all numbers from all series that have 4 digits.

In [1]:
def triangle(n):
    return n * (n + 1) // 2

def square(n):
    return n**2

def pentagonal(n):
    return n * (3*n - 1) // 2

def hexagonal(n):
    return n * (2 * n - 1)

def heptagonal(n):
    return n * (5 * n - 3) // 2

def octagonal(n):
    return n * (3 * n - 2)

In [2]:
def generate_numbers(series, min_num, max_num):
    n = 0
    ans = 0
    while ans <= max_num:
        n += 1
        ans = series(n) 
        if min_num <= ans <= max_num:
            yield ans
            

figurate_series_funcs = [triangle, square, pentagonal, hexagonal, heptagonal, octagonal]
figurate_series = [list(generate_numbers(series, 1000, 9999)) for series in figurate_series_funcs]

To make the computation faster, we compute just once *n mod 100* and *n // 100* (last two and first two digits) for each number.
Since we are looking for numbers that we can concatenate, we can exclude numbers whose last two digits are 01, 02, ..., 09.

For each series I created a dictionary that is accessible using the last two digits and returns a list of numbers inside the series that have those last two digits. For each number, the lists contains the number and the first two digits.

In [3]:
from collections import defaultdict

def to_dict(fig_s):
    
    figurate_series_dict = defaultdict(list)
    
    for n in fig_s:
        if n % 100 > 9:
            figurate_series_dict[n % 100].append((n, n // 100))
            
    return figurate_series_dict
    
figurate_series_to_dicts = [to_dict(fig_s) for fig_s in figurate_series]

Starting from the octagonal numbers (because there's less of them) I perform a tree search where I try to concatenate the numbers: the first two digits of the last concatenated number become the last two digits of the next number that I want to insert.

Notice that there is no constraint on the order of the series (it's not necessarily "triangle, then square, then pentagonal, ..."). I just have to make sure that for each series there is one and only one number in the collected tuple.

When a possible solution is found, I just need to check if the first and last number of the solution are concatenable. If yes, I print the numbers and their sum.

In [4]:
def check_valid_and_print(solution):
    if solution[0][0] % 100 == solution[-1][0] // 100:
        print [(value, figurate_series_funcs[idx_func].__name__) for value, idx_func in solution[::-1]]
        print 'Sum is:', sum(s[0] for s in solution)
    

def rec_search(fig_idx, mod_100, elements, idxs):
    
    updated_idxs = idxs[::]
    updated_idxs.remove(fig_idx)
    
    values_mod_100 = figurate_series_to_dicts[fig_idx].get(mod_100, None)
    if values_mod_100 is not None:
        for value, v_mod_100 in values_mod_100:
            updated_elements = elements[::]
            updated_elements.append((value, fig_idx))
            
            if len(updated_idxs) == 0:
                check_valid_and_print(updated_elements)
            else:
                for idx in updated_idxs:
                    rec_search(idx, v_mod_100, updated_elements, updated_idxs)


for key, values_list in figurate_series_to_dicts[-1].iteritems():
    for n, div_100 in values_list:
        rec_search(len(figurate_series_to_dicts) - 2, div_100, [(n, 5)], [0, 1, 2, 3, 4])

[(8128, 'hexagonal'), (2882, 'pentagonal'), (8256, 'triangle'), (5625, 'square'), (2512, 'heptagonal'), (1281, 'octagonal')]
Sum is: 28684
